## 循环神经网络(RNN)
使用TensorFlow构建一个循环神经网络(LSTM)
- 作者: Aymeric Damien
- 代码: https://github.com/aymericdamien/TensorFlow-Examples/

#### RNN概述

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" alt="nn" style="width: 600px;"/>

参考文献:
- [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf), Sepp Hochreiter & Jurgen Schmidhuber, Neural Computation 9(8): 1735-1780, 1997.

#### MNIST数据集简介
  
该例使用了MNIST手写数字数据集。MNIST数据集包含60000个实例，其中50000作为训练集，10000作为测试集。数字的大小已经被标准化和中心化到了固定的(0,1)区间(28\*28像素)。为了简便起见，每个图像矩阵被平铺，并转换为一个1维的numpy矩阵，其中包含784个特征点(28\*28)  
  
![MNIST 数据集](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)  
  
更多细节：http://yann.lecun.com/exdb/mnist/

In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

#### 1. 导入MNIST数据集

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/',one_hot = True) #当前目录下创建一个data文件夹即可

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


#### 2. 自定义参数

In [3]:
#训练参数
learning_rate = 0.001
training_step = 10000
batch_size = 128
display_step = 200

#神经网络模型参数
num_input = 28    #MNIST输入数据（图像28*28像素）
timesteps = 28 
num_hidden = 128  #隐层的特征数
num_classes = 10  #MNIST的类别数，0-9个数字

#TensorFlow Graph输入
X = tf.placeholder('float',[None,timesteps,num_input])
Y = tf.placeholder('float',[None,num_classes])

#自定义权重参数
weights = {
    'out':tf.Variable(tf.random_normal([num_hidden,num_classes]))
}
biases = {
    'out':tf.Variable(tf.random_normal([num_classes]))
}

#### 3. 构建模型

In [5]:
def RNN(x,weights,biases):
    
    #准备可以满足RNN要求的数据形式
    #现在数据的形式：(batch_size,timesteps,n_input)
    #需要的形式：'timestep'的一个Tenosr List：(batch_size,num_input)
    
    
    #展开来得到一个'timestep'的一个Tenosr List：(batch_size,num_input)
    x = tf.unstack(x,timesteps,1)
    
    #使用TensorFlow定义一个LSTM单元
    lstm_cell = rnn.BasicLSTMCell(num_hidden,forget_bias=1.0)
    
    #得到LSTM单元的输出
    outputs,states = rnn.static_rnn(lstm_cell,x,dtype = tf.float32)
    
    # 线性激活函数，使用RNN内循环的最后一个输出
    return tf.matmul(outputs[-1],weights['out']) + biases['out']


logits =  RNN(X,weights,biases)
prediction = tf.nn.softmax(logits)

#定义loss和optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits,
                                                                labels = Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate= learning_rate)
train_op = optimizer.minimize(loss_op)

#模型评估（使用了test logits，没有设置dropout）
correct_pred = tf.equal(tf.argmax(prediction,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

#初始化变量
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



#### 4. 训练模型

In [7]:
#开始训练
with tf.Session() as sess:
    
    #初始化
    sess.run(init)
    
    for step in range(1,training_step+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #将数据变形为28个序列，每个序列含有28个元素
        batch_x = batch_x.reshape((batch_size,timesteps,num_input))
        u87
        sess.run(train_op, feed_dict = {X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            #计算loss和accuracy
            loss, acc = sess.run([loss_op,accuracy],feed_dict = {X:batch_x,
                                                                 Y:batch_y})
            
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
    print('训练结束!')
    
    
    #在128张测试集照片中计算accuracy
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1,timesteps,num_input))
    test_label = mnist.test.labels[:test_len]
    print ('Testing Accuracy:',  
          sess.run(accuracy,feed_dict = {X:test_data,Y:test_label}))

Step 1, Minibatch Loss= 2.8463, Training Accuracy= 0.141
Step 200, Minibatch Loss= 2.1587, Training Accuracy= 0.227
Step 400, Minibatch Loss= 2.0192, Training Accuracy= 0.328
Step 600, Minibatch Loss= 1.8719, Training Accuracy= 0.391
Step 800, Minibatch Loss= 1.7346, Training Accuracy= 0.469
Step 1000, Minibatch Loss= 1.6994, Training Accuracy= 0.500
Step 1200, Minibatch Loss= 1.4999, Training Accuracy= 0.594
Step 1400, Minibatch Loss= 1.5138, Training Accuracy= 0.570
Step 1600, Minibatch Loss= 1.3649, Training Accuracy= 0.609
Step 1800, Minibatch Loss= 1.3557, Training Accuracy= 0.570
Step 2000, Minibatch Loss= 1.2730, Training Accuracy= 0.570
Step 2200, Minibatch Loss= 1.3521, Training Accuracy= 0.570
Step 2400, Minibatch Loss= 1.2243, Training Accuracy= 0.617
Step 2600, Minibatch Loss= 1.1484, Training Accuracy= 0.648
Step 2800, Minibatch Loss= 1.1261, Training Accuracy= 0.641
Step 3000, Minibatch Loss= 0.9713, Training Accuracy= 0.711
Step 3200, Minibatch Loss= 1.0195, Training Acc